# **Deteccion de fraude crediticio con machine learning** (Ricardo Bustos Carreón)
Partimos de un conjunto de datos con 7300 entradas y 31 variables de las cuales 29 son desconocidas, dos conocidas y el tipo de variable. De estas entradas 7000 son transacciones normales y 300 son transacciones fraudulentas, nuestro proposito es aprender a identificar si una operacion es legal o fraudulenta dado que este tipo de operaciones cuestan tanto al cliente como a la empresa, por eso comenzamos con un analisis enfocado el machine learning.

## **Analisis exploratorio de datos**
Ya conocemos nuestras variables, procederemos al analisis. No podemos basar si una transaccion es fraudulenta en funcion de la media o maxima pues la distribución del valor monetario de todas las transacciones está muy sesgada. La gran mayoría de las transacciones son relativamente pequeñas y solo una pequeña fracción de las transacciones se acerca al máximo.

Ahora, ¿Qué pasa con las distribuciones de clase? ¿Cuántas transacciones son fraudulentas y cuántas no lo son? Bueno, como podemos esperar, la mayoría de las transacciones no son fraudulentas (95.890%), mientras que solo el 4.110% fueron fraudulentas.
Visualicemoslo ahora con nuestro algoritmo: 


In [1]:
import scipy.stats as stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

df = pd.read_csv('C:/Users/ricar/Desktop/dataset.csv')
print('El data frame tiene {} filas y {} columnas.'.format(df.shape[0], df.shape[1]))
print (df.sample(5))
print(df.info())

df.loc[:, ['Var', 'Amount']].describe()
#visualizacions de Var y amount
plt.figure(figsize=(10,8))
plt.title('Distribucion de la variable (Var)')
sns.distplot(df.Var)


El data frame tiene 7300 filas y 31 columnas.
         Var        V1        V2        V3  ...       V27       V28  Amount  Class
1652  105397  1.287631  0.410746  0.151841  ...  0.030363  0.020647    6.95      0
4902  129463  1.454601 -1.063245  0.440964  ...  0.021222  0.011972   25.00      0
2568   15877  1.219136  0.535374 -0.482669  ... -0.041098  0.032509    0.76      0
947   169560 -1.317357  1.609457 -1.498172  ...  0.273136 -0.035843    8.97      0
839   233197  2.113497  0.575076 -2.803749  ...  0.023942 -0.002685    1.00      0

[5 rows x 31 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7300 entries, 0 to 7299
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Var     7300 non-null   int64  
 1   V1      7300 non-null   float64
 2   V2      7300 non-null   float64
 3   V3      7300 non-null   float64
 4   V4      7300 non-null   float64
 5   V5      7300 non-null   float64
 6   V6      7300 non-null   f

In [1]:
plt.figure(figsize=(10,8))
plt.title('Distribucion del valor monetario (Amount)')
sns.distplot(df.Amount)


<h3 align='center'><img src='https://scontent.fmex5-1.fna.fbcdn.net/v/t39.30808-6/315073115_3266202986977681_5264328340428307278_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=730e14&_nc_eui2=AeHCI8zgQYdVlxaEPk_AKdCDOrmmh0a7JBA6uaaHRrskEDfKjKASta6857Q_LXz_f4a5VC5Ybgeq0s-UX0JQ-YaT&_nc_ohc=0cHt4AVji-oAX_jC2Z7&tn=iAnTb-G2baG76wTk&_nc_ht=scontent.fmex5-1.fna&oh=00_AfBkv1HGDFGEjdIcBsYaIPjchSRln8mzP1eV1sBzvovh9Q&oe=636F3A26' alt='python' width='400' height='400'>


In [1]:
#Comparacion de transacciones fraudulentas contra normales
counts = df.Class.value_counts()
normal = counts[0]
fraudulent = counts[1]
perc_normal = (normal/(normal+fraudulent))*100
perc_fraudulent = (fraudulent/(normal+fraudulent))*100
print('Tenemos {} transacciones normales ({:.3f}%) y {} transacciones fraudulentas ({:.3f}%).'.format(normal, perc_normal, fraudulent, perc_fraudulent))
plt.figure(figsize=(8,6))
sns.barplot(x=counts.index, y=counts)
plt.title('Transacciones normales contra fraudulentas')
plt.ylabel('Count')
plt.xlabel('Class (0:Normales, 1:Fraudulentas)')


Tenemos 7000 transacciones normales (95.890%) y 300 transacciones fraudulentas (4.110%).
<h3 align='center'><img src='https://scontent.fmex5-1.fna.fbcdn.net/v/t39.30808-6/314777348_3266202960311017_8811628485310736382_n.jpg?_nc_cat=102&ccb=1-7&_nc_sid=730e14&_nc_eui2=AeFndORm5pymB2H0mDyIuESrA2ZO7l_wIfUDZk7uX_Ah9YlVZz5OkRnKA7W0bqm4SK-eXg7RWcaIKwTJ_JpFqn1v&_nc_ohc=C8ZaVes1tUAAX8_UmA4&_nc_ht=scontent.fmex5-1.fna&oh=00_AfA5Ww79myRSh-6MYw045ZgwkqAVp4j2yowZ2iPnPvnMvg&oe=636EA2DE' alt='python' width='400' height='400'>


Observamos ahora la correlacion entre nuestras variables, para este caso usamos un mapa de calor, podemos apreciar una fuerte correlacion entre nuestras variables y las variables de clase.
Visualicemos nuestro algoritmo: 


In [1]:
corr = df.corr()
print (corr)
#Mapa de calor para visualización
corr = df.corr()
plt.figure(figsize=(12,10))
heat = sns.heatmap(data=corr)
plt.title('Mapa de calor para correlacion')
#Simetria
skew_ = df.skew()
print (skew_)


             Var        V1        V2  ...       V28    Amount     Class
Var     1.000000  0.139439 -0.038180  ... -0.000669 -0.009194 -0.059608
V1      0.139439  1.000000 -0.326759  ... -0.056911 -0.208454 -0.391060
V2     -0.038180 -0.326759  1.000000  ... -0.052532 -0.501140  0.372244
V3     -0.185765  0.484560 -0.408068  ... -0.039208 -0.147528 -0.569067
V4     -0.125990 -0.279497  0.266135  ...  0.053758  0.095444  0.512064
V5      0.184853  0.457110 -0.290087  ... -0.048134 -0.298085 -0.380176
V6     -0.032212  0.089315 -0.135573  ...  0.062639  0.204001 -0.194888
V7      0.116759  0.486588 -0.470636  ...  0.072419  0.247992 -0.527325
V8     -0.069866 -0.071541  0.031140  ... -0.033491 -0.068391  0.093921
V9      0.050420  0.263985 -0.250592  ...  0.035151 -0.024197 -0.399792
V10     0.075346  0.420311 -0.359026  ...  0.020854 -0.070770 -0.622983
V11    -0.230156 -0.300984  0.282850  ...  0.028729  0.016035  0.559960
V12     0.130645  0.402597 -0.376569  ... -0.020242 -0.000617 -0

# **Deteccion de fraude crediticio con machine learning** (Ricardo Bustos Carreón)
Partimos de un conjunto de datos con 7300 entradas y 31 variables de las cuales 29 son desconocidas, dos conocidas y el tipo de variable. De estas entradas 7000 son transacciones normales y 300 son transacciones fraudulentas, nuestro proposito es aprender a identificar si una operacion es legal o fraudulenta dado que este tipo de operaciones cuestan tanto al cliente como a la empresa, por eso comenzamos con un analisis enfocado el machine learning.

## **Escalado de Var y Amount**


In [1]:
#Escalado de cantidad (Amount) y (Var)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler2 = StandardScaler()
#scalando Var
scaled_time = scaler.fit_transform(df[['Var']])
flat_list1 = [item for sublist in scaled_time.tolist() for item in sublist]
scaled_time = pd.Series(flat_list1)
#scalando la columnda de cantidad (amount)
scaled_amount = scaler2.fit_transform(df[['Amount']])
flat_list2 = [item for sublist in scaled_amount.tolist() for item in sublist]
scaled_amount = pd.Series(flat_list2)
#Concatenamos las columnas nuevas y el dataframe original
df = pd.concat([df, scaled_amount.rename('scaled_amount'), scaled_time.rename('scaled_time')], axis=1)
print(df.sample(5))
#quitamos las columnas de amount y Var 
df.drop(['Amount','Var'], axis=1, inplace=True)


         Var        V1        V2  ...  Class  scaled_amount  scaled_time
5457  152309  2.051476 -0.328346  ...      0      -0.255451     0.131658
3407  280439  2.035164 -0.139755  ...      0      -0.309429     1.686554
441    79993 -1.710418  0.382147  ...      0      -0.309049    -0.745918
1755  171772  2.199101 -0.795777  ...      0      -0.209114     0.367848
7145   10497  1.189784  0.942289  ...      1      -0.299401    -1.589273

[5 rows x 33 columns]


## **Creando datos de entrenamiento y test** 
Para resolver este punto nos enfocaremos en realizar un muestreo inferior aleatorio para crear un conjunto de datos de entrenamiento con una distribución de clases equilibrada, asi obligaremos a los algoritmos a detectar transacciones fraudulentas como tales para lograr un alto rendimiento.

Refiriendonos al rendimiento, vamos a hacer uso de las características de funcionamiento del receptor: área bajo la curva o medida de rendimiento ROC-AUC, escencialmente produce un valor entre cero y uno, por lo que uno es un puntaje perfecto y cero el peor. Entonces, si un algoritmo tiene un puntaje de más de 0.5, está logrando un mejor desempeño que las conjeturas al azar.
Visualicemos nuestro algoritmo: 


In [1]:
#Separando informacion en train y test
mask = np.random.rand(len(df)) < 0.9
train = df[mask]
test = df[~mask]
print('Entrenamiento: {}
Test: {}'.format(train.shape, test.shape))
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)


In [1]:
#Creamos un subejemplo con distribuciones de clase balanceadas

no_of_frauds = train.Class.value_counts()[1]
print('Tenemos {} transacciones fraudulentas en la data de entrenamiento.'.format(no_of_frauds))
#seleccionamos la cantidad equivalente de transacciones normales
non_fraud = train[train['Class'] == 0]
fraud = train[train['Class'] == 1]
selected = non_fraud.sample(no_of_frauds)
print(selected.head())
#concatenamos ambas en una data de ejemplocon distribuciones de clase equivalente
selected.reset_index(drop=True, inplace=True)
fraud.reset_index(drop=True, inplace=True)
subsample = pd.concat([selected, fraud])
len(subsample)
subsample = subsample.sample(frac=1).reset_index(drop=True)
subsample.head(10)
new_counts = subsample.Class.value_counts()
plt.figure(figsize=(8,6))
sns.barplot(x=new_counts.index, y=new_counts)
plt.title('Numero de transacciones fraudulentas contra normales en el ejemplo')
plt.ylabel('Count')
plt.xlabel('Class (0:Normales, 1:Fraudulentas)')


Entrenamiento: (6547, 31)
Test: (753, 31)
Tenemos 267 transacciones fraudulentas en la data de entrenamiento.
            V1        V2        V3  ...  Class  scaled_amount  scaled_time
1201 -0.370695  0.817413  1.500371  ...      0      -0.269939    -1.687617
3452  2.005876 -0.403021 -1.211072  ...      0      -0.213090     1.413304
5189 -0.937393  0.710026 -0.237297  ...      0      -0.242068     0.673147
2647  1.189597 -0.042003 -0.301652  ...      0      -0.050047    -0.144068
1925  1.240280  0.002590 -1.331894  ...      0      -0.190925    -1.176442

[5 rows x 31 columns]
<h3 align='center'><img src='https://scontent.fmex5-1.fna.fbcdn.net/v/t39.30808-6/314846788_3266203073644339_2347546527326195984_n.jpg?_nc_cat=111&ccb=1-7&_nc_sid=730e14&_nc_eui2=AeHNLdZZIp1rWCZE2ne5JC4aooqy7jl6Iz6iirLuOXojPvEeaB_4LbS3AYS4QzLRISPi18m5QbjqtTKqPTcsukpz&_nc_ohc=Ncy57aAVe6kAX-Pzt67&_nc_ht=scontent.fmex5-1.fna&oh=00_AfCWmFqht2epNuyJbNH3XvoQ2WH5DDbTeaYQs6uTU5p1EQ&oe=636E9FF9' alt='python' width='400' he

In [1]:
#Veamos la correlacion una vez mas
corr = subsample.corr()
corr = corr[['Class']]
print(corr)


                  Class
V1            -0.435720
V2             0.495217
V3            -0.549128
V4             0.716220
V5            -0.415188
V6            -0.384172
V7            -0.491535
V8             0.083490
V9            -0.516827
V10           -0.613859
V11            0.680231
V12           -0.666235
V13           -0.027206
V14           -0.738973
V15           -0.003168
V16           -0.599844
V17           -0.566460
V18           -0.479347
V19            0.331789
V20            0.125046
V21            0.149468
V22            0.014182
V23            0.030585
V24            0.044720
V25            0.010432
V26            0.032597
V27            0.032605
V28            0.130370
Class          1.000000
scaled_amount  0.008548
scaled_time   -0.160106


In [1]:
#correlaciones negativas menores a -0.5
print(corr[corr.Class < -0.5])


        Class
V3  -0.549128
V9  -0.516827
V10 -0.613859
V12 -0.666235
V14 -0.738973
V16 -0.599844
V17 -0.566460


In [1]:
#correlaciones positivas mayores a 0.5
print(corr[corr.Class > 0.5])


          Class
V4     0.716220
V11    0.680231
Class  1.000000


In [1]:
#visualizando las variables con alta correlacion negativa
f, axes = plt.subplots(nrows=2, ncols=4, figsize=(26,16))

f.suptitle('variables con alta correlacion negativa', size=35)
sns.boxplot(x='Class', y='V3', data=subsample, ax=axes[0,0])
sns.boxplot(x='Class', y='V9', data=subsample, ax=axes[0,1])
sns.boxplot(x='Class', y='V1', data=subsample, ax=axes[0,2])
sns.boxplot(x='Class', y='V1', data=subsample, ax=axes[0,3])
sns.boxplot(x='Class', y='V1', data=subsample, ax=axes[1,0])
sns.boxplot(x='Class', y='V1', data=subsample, ax=axes[1,1])
sns.boxplot(x='Class', y='V1', data=subsample, ax=axes[1,2])
f.delaxes(axes[1,3])


<h3 align='center'><img src='https://scontent.fmex5-1.fna.fbcdn.net/v/t39.30808-6/314910474_3266203113644335_8104210165392035790_n.jpg?_nc_cat=107&ccb=1-7&_nc_sid=730e14&_nc_eui2=AeFaUQfvLJGQawdLqzRQoO7ycsH8Sn1PIT1ywfxKfU8hPVFXE7NGS_IE83KpuOuMUm2gFHF1IjEHTJNcTq9sMxZZ&_nc_ohc=7tGaDJqw2rAAX83Bj2W&_nc_ht=scontent.fmex5-1.fna&oh=00_AfA8ATpTuC5iAY_MtYuybM-pMltIrcCkSZp3TZUFNsP80w&oe=636DBD82' alt='python' width='400' height='400'>


In [1]:
#visualizando las variables con alta correlacion positiva
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(18,9))

f.suptitle('variables con alta correlacion positiva', size=20)
sns.boxplot(x='Class', y='V4', data=subsample, ax=axes[0])
sns.boxplot(x='Class', y='V11', data=subsample, ax=axes[1])


<h3 align='center'><img src='https://scontent.fmex23-1.fna.fbcdn.net/v/t39.30808-6/315003758_3266203110311002_8851833949373281310_n.jpg?_nc_cat=100&ccb=1-7&_nc_sid=730e14&_nc_eui2=AeH9gNpN3Kg5fLuKrKBXlAtPifWoE2FrEcyJ9agTYWsRzIPtHEs_DA_Ta6n6uXxjY0-p_7Vq7Hv0gabF9SXnb87Q&_nc_ohc=_BePVt0XNMsAX_eC1wu&_nc_ht=scontent.fmex23-1.fna&oh=00_AfA7lrnXUWwIbRnaNKO5X3xWbM_1DFXTKJ28u3xbQS6lDw&oe=636DE971' alt='python' width='400' height='400'>


Terminamos con la eliminacion de variables atipicas, la reduccion de dimensionalidad y los algoritmos de clasificacion


In [2]:
#Eliminacion de variables atipicas extremas

#Removiendo resultados atipicos
Q1 = subsample.quantile(0.25)
Q3 = subsample.quantile(0.75)
IQR = Q3 - Q1

df2 = subsample[~((subsample < (Q1 - 2.5 * IQR)) |(subsample > (Q3 + 2.5 * IQR))).any(axis=1)]
len_after = len(df2)
len_before = len(subsample)
len_difference = len(subsample) - len(df2)
print('Reducimos nuestra data de {} transacciones a {} transacciones a {} transacciones.'.format(len_before, len_difference, len_after))

#Reduccion de dimensionalidad

from sklearn.manifold import TSNE

X = df2.drop('Class', axis=1)
y = df2['Class']
#t-SNE
X_reduced_tsne = TSNE(n_components=2, random_state=42).fit_transform(X.values)
# t-SNE scatter plot
import matplotlib.patches as mpatches
f, ax = plt.subplots(figsize=(24,16))
blue_patch = mpatches.Patch(color='#0A0AFF', label='Normal')
red_patch = mpatches.Patch(color='#AF0000', label='Fraude')

ax.scatter(X_reduced_tsne[:,0], X_reduced_tsne[:,1], c=(y == 0), cmap='coolwarm', label='No Fraud', linewidths=2)
ax.scatter(X_reduced_tsne[:,0], X_reduced_tsne[:,1], c=(y == 1), cmap='coolwarm', label='Fraud', linewidths=2)
ax.set_title('t-SNE', fontsize=14)
ax.grid(True)
ax.legend(handles=[blue_patch, red_patch])

#Algoritmos de clasificacion 

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
# train, test, split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.values
X_validation = X_test.values
y_train = y_train.values
y_validation = y_test.values
print('X_shapes:
', 'X_train:', 'X_validation:
', X_train.shape, X_validation.shape, '
')
print('Y_shapes:
', 'Y_train:', 'Y_validation:
', y_train.shape, y_validation.shape)

#Algoritmos de verificacion lineal
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
models = []
models.append(('LR', LogisticRegression())) #Regresion logistica
models.append(('LDA', LinearDiscriminantAnalysis())) #Linear Discriminant Analysis
models.append(('KNN', KNeighborsClassifier())) #KNN
models.append(('CART', DecisionTreeClassifier())) #Decision tree
models.append(('RF', RandomForestClassifier())) #Random forest
#probando modelos
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='roc_auc')
    results.append(cv_results)
    names.append(name)
    msg = '%s: %f (%f)' % (name, cv_results.mean(), cv_results.std())
    print(msg)
#Comparando Algoritmos
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(1,1,1)
ax.set_xticklabels(names)
plt.title('Comparacion de algoritmos de clasificacion')
plt.xlabel('Algoritmo')
plt.ylabel('ROC-AUC Puntaje')
plt.boxplot(results)
plt.show()


Reducimos nuestra data de 534 transacciones a 154 transacciones a 380 transacciones.
X_shapes:
 X_train: X_validation:
 (304, 30) (76, 30) 

Y_shapes:
 Y_train: Y_validation:
 (304,) (76,)
LR: 0.947495 (0.048584)
LDA: 0.957383 (0.030403)
KNN: 0.935032 (0.060732)
CART: 0.862575 (0.060110)
RF: 0.953353 (0.040278)
<h3 align='center'><img src='https://scontent.fmex23-1.fna.fbcdn.net/v/t39.30808-6/315001427_3266203156977664_6113497219873285270_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=730e14&_nc_eui2=AeHZP5Go_xHqFJStX9VR2tbGZbDsyqtcBCNlsOzKq1wEI5f1RnuYiz_OAW0QqyCNp8eQxuF35x5VFknZ28XFKyZF&_nc_ohc=tah4Dko4bqQAX-J0Fk3&_nc_ht=scontent.fmex23-1.fna&oh=00_AfDhC0T2Ifq1FblKfGa1hfpwSgAdvF3UDXggKGz1syl_4A&oe=636EEFCC' alt='python' width='400' height='400'>
<h3 align='center'><img src='https://scontent.fmex5-1.fna.fbcdn.net/v/t39.30808-6/314950798_3266202990311014_3964461776007888216_n.jpg?_nc_cat=102&ccb=1-7&_nc_sid=730e14&_nc_eui2=AeEbiilC5k2GRFi8262-540gF8SK3fhQaLEXxIrd-FBosRQI1ULMC_QfIbpt57mgRP7BTVy4dlfmr